# Peak routing

_Contributed by Dr. Stephan Fuchs_

In this tutorial we will:
- extract some mountain peaks from OpenStreetMap using the `overpy` package
- calculate the routes to the peaks from different starting points
- save the maps as PNG files using an intermediate HTML representation
as well as the `selenium` and `webdriver-manager` packages

#### Create an output folder for the maps if it doesn't exist already

In [1]:
import os

map_folder = f"{os.getcwd()}/maps"

if not os.path.exists(map_folder):
    print("Creating maps folder")
    os.makedirs(map_folder)

#### Get the coordinates of the peaks
You can directly download the peak data via the overpy package.
It uses the same query language as [overpass-turbo](https://overpass-turbo.eu/#).

Here the peaks where the name matches one of the `peak_names` will be extracted from the given bounding box.

In [2]:
import overpy
import json

api = overpy.Overpass()

bbox = [50.82263785103416, 13.996753692626951, 50.843670947425615, 14.116744995117186]
peak_names = ["Zeisigstein", "Herkuleskopf", "Kristin Hrádek"]

# Query natural peaks from overpass api within bounding box
result = api.query(
    f"""node["natural"="peak"][name~"^({"|".join(peak_names)})$"]({",".join([str(c) for c in bbox])});out;""")

# Extract name and coordinates
peaks = [{"name": n.tags['name'], "LonLat": [float(n.lon), float(n.lat)]} for n in result.nodes]
print(json.dumps(peaks, indent=2))

[
  {
    "name": "Herkuleskopf",
    "LonLat": [
      14.0429261,
      50.8422103
    ]
  },
  {
    "name": "Zeisigstein",
    "LonLat": [
      13.9996773,
      50.8241813
    ]
  },
  {
    "name": "Kristin Hr\u00e1dek",
    "LonLat": [
      14.1151461,
      50.8263686
    ]
  }
]


#### The starting points
As starting locations we choose one bus station, a parking place and a Hotel from the surrounding area.
For simplicity, the coordinates are given already.
An identification color is added for styling corresponding elements.

In [3]:
start_points = [
    {
        "name": "Bushaltestelle Zollhaus",
        "LonLat": [14.067741, 50.832176],
        "color": "blue"
    }, {
        "name": "Waldparkplatz Ottomühle",
        "LonLat": [14.045109, 50.839162],
        "color": "red"
    }, {
        "name": "Hotel Ostrov",
        "LonLat": [14.046037, 50.803897],
        "color": "orange"
    }
]

#### Calculate the routes
For each of the peaks we create a folium map with routes from every starting point, and add it to the `maps` list.
To every map we add the points, the calculated routes and a little styling.

__Make sure to insert a valid openrouteservice API key before running the cell__

In [4]:
import folium
import openrouteservice

client = openrouteservice.Client(key="your_api_key")


# The style for the plotted routes.
def route_style(color):
    return lambda feature: dict(color=color, opacity=0.9, weight=4, )


maps = []
for peak in peaks:
    # initialize map and storages
    map_object = folium.Map()
    bounds = []
    routes = {}

    for start in start_points:
        # Add point markers with LatLon coordinates
        folium.Marker(list(reversed(start["LonLat"])), popup=start["name"],
                      icon=folium.Icon(icon_color=start["color"], icon='taxi', prefix='fa')).add_to(map_object)
        folium.Marker(list(reversed(peak["LonLat"])), popup=peak["name"],
                      icon=folium.Icon(icon="chevron-up", icon_color="dark-grey", prefix="fa")).add_to(map_object)

        # request walking route from start to peak
        response = client.directions([start["LonLat"], peak["LonLat"]], profile='foot-walking', geometry='true',
                                     format_out="geojson")

        # extract route info
        routes[start["name"]] = {
            "distance_m": round(response["features"][0]["properties"]["summary"]["distance"]),
            "duration_min": round(response["features"][0]["properties"]["summary"]["duration"] / 60.)
        }

        # add styled route objects
        route = folium.GeoJson(response, style_function=route_style(start["color"]))
        route.add_to(map_object)

        # add bounds to list for overview-bounds calculation
        bounds.append(route.get_bounds())

    # calculate overview-bounds
    bbox = [
        [min(c[0][0] for c in bounds),
         min(c[0][1] for c in bounds)],
        [max(c[1][0] for c in bounds),
         max(c[1][1] for c in bounds)]
    ]
    map_object.fit_bounds(bbox)
    maps.append({
        "map_obj": map_object,
        "routes": routes
    })
    del routes

    # save as html to maps folder
    map_object.save("maps/" + peak["name"] + ".html")

#### Display maps
Show route information and map interface (Only one map can be shown per Cell).

In [5]:
def print_map_info(routes_dict: dict):
    """
    Prints info for each start point
    """
    for name, summary in routes_dict.items():
        print(f"From {name}: distance {summary['distance_m']} m, duration: {summary['duration_min']} min")

In [6]:
print(f"-- Routes to {peak_names[0]} peak --")
print_map_info(maps[0]["routes"])
maps[0]["map_obj"]

-- Routes to Zeisigstein peak --
From Bushaltestelle Zollhaus: distance 2857 m, duration: 34 min
From Waldparkplatz Ottomühle: distance 532 m, duration: 6 min
From Hotel Ostrov: distance 5455 m, duration: 65 min


In [7]:
print(f"-- Routes to {peak_names[1]} peak --")
print_map_info(maps[1]["routes"])
maps[1]["map_obj"]

-- Routes to Herkuleskopf peak --
From Bushaltestelle Zollhaus: distance 6932 m, duration: 83 min
From Waldparkplatz Ottomühle: distance 4673 m, duration: 56 min
From Hotel Ostrov: distance 4854 m, duration: 58 min


In [8]:
print(f"-- Routes to {peak_names[2]} peak --")
print_map_info(maps[2]["routes"])
maps[2]["map_obj"]


-- Routes to Kristin Hrádek peak --
From Bushaltestelle Zollhaus: distance 4601 m, duration: 55 min
From Waldparkplatz Ottomühle: distance 6117 m, duration: 73 min
From Hotel Ostrov: distance 6503 m, duration: 78 min


#### Save maps as PNG
For transformation to PNG, we need a browser to take a screenshot from.
Luckily `webdriver-manager` takes care of installing the browser driver for us.

After opening the images with the webdriver, we can set the resolution and save a screenshot of the browser window, to
get our desired PNG file.

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_driver.set_window_size(1280, 900)  # choose a resolution

for peak in peak_names:
    path = f"file://{map_folder}/{peak}.html"
    chrome_driver.get(url=path)
    time.sleep(2)  # Waiting for page loading
    chrome_driver.save_screenshot(f"{map_folder}/{peak}.png")

chrome_driver.quit()



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/amandusbutzer/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


#### Additional image manipulation (optional)
In case the image needs to be processed further e.g. the `Pillow` package can be used to manipulate it.
Here a simple resize is shown to reduce the resolution while keeping the aspect ratio.

In [10]:
from PIL import Image

for peak in peak_names:
    image = Image.open(f"{map_folder}/{peak}.png")
    new_image = image.resize([int(i * 0.25) for i in image.size])  # resize to 1/4 the resolution
    new_image.save(f"{map_folder}/{peak}_small.png")